# 基于用户的协同过滤算法
<a id="org51d1b05"></a>
## 基础算法

在一个在线个性化推荐系统中，当一个用户A需要个性化推荐时，可以先找到和他有相似兴趣的其他用户，然后把那些用户喜欢的、而用户A没有听说过的物品推荐给A。这种方法称为基于用户的协同过滤算法。

主要包括两个步骤。


<a id="orgae2fd56"></a>
### 找到和目标用户兴趣相似的用户集合

利用行为的相似度计算兴趣的相似度。

给定用户 $u$ 和用户 $v$ ，令 $N(u)$ 表示用户 $u$ 曾经有过正反馈的物品集合，令 $N(v)$ 为用户 $v$ 曾经有过正反馈的物品集合。通过如下的Jaccard公式<sup><a id="fnr.1" class="footref" href="#fn.1">1</a></sup>简单地计算 $u$ 和 $v$ 的兴趣相似度：

\begin{equation}
w_{uv} = \frac{| N (u) \cap N (v) |}{| N (u) \cup N (v) | } \nonumber
\end{equation}

或者通过余弦相似度<sup><a id="fnr.2" class="footref" href="#fn.2">2</a></sup>计算<sup><a id="fnr.3" class="footref" href="#fn.3">3</a></sup>：

\begin{equation}
w_{uv} = \frac{| N (u) \cap N (v) |}{\sqrt{| N (u) | | N (v) | }} \nonumber
\end{equation}


比如以下面的用户行为记录为例，举例说明 **UserCF** 计算用户兴趣相似度的例子。
用户行为记录举例：

    A -> [a] [b] [d]
    B -> [a] [c]
    C -> [b] [e]
    D -> [c] [d] [e]

用户 $A$ 对物品 ${a, b, d}$ 有过行为，用户 $B$ 对物品 ${a, c}$ 有过行为，利用余弦相似度公式计算用户 $A$ 和用户 $B$ 的兴趣相似度为：

\begin{equation}
w_{AB} = \frac{| \{a, b, d\} \cap \{a, c\} |}{\sqrt{| \{a, b, d\} | | \{a, c\} |}} = \frac{1}{\sqrt{6}} \nonumber
\end{equation}

同理可以计算出用户 $A$ 和用户 $C$ 、 $D$ 的相似度：

\begin{equation}
w_{AC} = \frac{| \{a, b, d\} \cap \{b, e\} |}{\sqrt{| \{a, b, d\} | | \{b, e\} |}} = \frac{1}{\sqrt{6}} \nonumber
\end{equation}

\begin{equation}
w_{AD} = \frac{| \{a, b, d\} \cap \{c, d, e\} |}{\sqrt{| \{a, b, d\} | | \{c, d, e\} |}} = \frac{1}{3} \nonumber
\end{equation}


实现余弦相似度的伪代码：
```python
def user_similarity(train):
    import math

    W = dict()
    for u in train.keys():
        for v in train.keys():
            if u == v:
                continue
            W[u][v] = len(train[u] & train[v])
            W[u][v] = W[u][v] / math.sqrt(len(train[u]) * len(train[v]))
    return W
```

上述代码对两两用户都利用余弦相似度计算相似度。方法的时间复杂度是 $O(|U|\times|U|)$ ，在用户数很大时非常耗时。

**事实上，很多用户相互之间并没有对同样的物品产生过行为** ，即很多时候 $|N(u) \cap N(v)|=0$ 。

可以先计算出 $|N(u) \cap N(v)| \neq 0$ 的用户对 $(u,v)$ ，再对这种情况除以分母 $\sqrt{|N(u)||N(v)|}$ 。

建立物品到用户的倒排表，对于每个物品都保存对该物品产生过行为的用户列表。

令稀疏矩阵 $C[u][v] = |N(u) \cap N(v)|$ ，假设用户 $u$ 和用户 $v$ 同时属于倒排表中 $K$ 个物品对应的用户列表，则 $C[u][v]=K$ 。可以扫描倒排表中每个物品对应的用户列表，将用户列表中的两两用户对应的 $C[u][v]$ **加1** 。最终得到所有用户之间不为0的 $C[u][v]$ 。

相应的伪码：

```python
def user_similarity(train):
    import math

    # build inverse table for item_users
    item_users = dict()
    for u, items in train.items():
        for i in items.keys():
            if i not in item_users:
                item_users[i] = set()
            item_users[i].add(u)

    # calculate co-related items between users
    C = dict()
    N = dict()
    for i, users in item_users.items():
        for u in users:
            N[u] += 1
            for v in users:
                if u == v:
                    continue
                C[u][v] += 1

    # calculate final similarity matrix W
    W = dict()
    for u, related_users in C.items():
        for u, cuv in related_users.items():
            W[u][v] = cuv / math.sqrt(N[u] * N[v])
    return W
```



<a id="org642930e"></a>
### 找到这个集合中的用户喜欢的，且目标用户没有听说过的物品推荐给目标用户

得到用户之间的兴趣相似度后， **UserCF** 算法会给用户推荐和他兴趣最相似的 $K$ 个用户喜欢的物品。

下面的公式度量了 **UserCF** 算法中用户 $u$ 对物品 $i$ 的感兴趣程度：

\begin{equation}
p(u,i) = \sum_{v \in S(u,K) \cap N(i)} w_{uv}r_{vi} \nonumber
\end{equation}

其中， $S(u,K)$ 包含和用户 $u$ 兴趣最接近的 $K$ 个用户， $N(i)$ 是对物品 $i$ 有过行为的用户集合， $w_{uv}$ 是用户 $u$ 和用户 $v$ 的兴趣相似度， $r_{vi}$ 代表用户 $v$ 对物品 $i$ 的兴趣。因为使用的是单一行为的隐反馈数据，所以所有的 $r_{vi}=1$ 。

下面的伪码实现了上面的 **UserCF** 算法：

```python
def recommend(user, train, W, K):
    rank = dict()
    interacted_items = train[user]
    for v, wuv in sorted(W[u].items, key=itemgetter(1), reverse=True)[0:K]:
        for i, rvi in train[v].items:
            if i in interacted_items:
                # we should filter items user interacted before
                continue
            rank[i] += wuv * rvi
    return rank
```


<a id="org83f46c3"></a>
## 用户相似度的改进 **User-IIF** 算法

两个用户对冷门物品采取过同样的行为更能说明他们兴趣的相似度。

John S. Breese在论文<sup><a id="fnr.4" class="footref" href="#fn.4">4</a></sup>中提出的根据用户行为计算用户的兴趣相似度：

\begin{equation}
w_{uv} = \frac{\sum_{i \in N (u) \cap N (v)} \frac{1}{\log{(1 + | N (i) |)}}}{\sqrt{| N(u) | | N(v) |}} \nonumber
\end{equation}

该公式通过 $\frac{1}{\log{(1 + | N (i) |)}}$ 惩罚了用户 $u$ 和用户 $v$ 共同兴趣列表中热门物品对他们相似度的影响。

```python
def user_similarity(train):
    import math

    # build inverse table for item_users
    item_users = dict()
    for u, items in train.items():
        for i in items.keys():
            if i not in item_users:
                item_users[i] = set()
            item_users[i].add(u)

    # calculated co-rated items between users
    C = dict()
    N = dict()
    for i, users in item_users.items():
        for u in users:
            N[u] += 1
            for v in users:
                if u == v:
                    continue
                C[u][v] += 1 / math.log(1 + len(users))

    # calculate final similarity matrix W
    W = dict()
    for u, related_users in C.items():
        for v, cuv in related_users.items():
            W[u][v] = cuv / math.sqrt(N[u] * N[v])
    return W
```


<a id="org787bce1"></a>

## 缺点

-   随着网站用户数的增大，计算用户兴趣相似度矩阵越发困难，其运算时间复杂度和空间复杂度的增长和用户数的增长近似于平方关系。

-   很难对推荐结果作出解释。


## 脚注

<sup><a id="fn.1" class="footnum" href="#fnr.1">1</a></sup> 中文常翻译为“杰卡德公式”，公式如下：

\begin{equation}
J(A,B) = \frac{|A \cap B|}{| A \cup B|} = \frac{| A \cap B|}{|A| + |B| - |A \cap B|} \nonumber
\end{equation}

<sup><a id="fn.2" class="footnum" href="#fnr.2">2</a></sup> 假定 $A$ 和 $B$ 是两个 $n$ 维向量， $A$ 是 $[A_1, A_2, \ldots, A_n]$ ， $B$ 是 $[B_1, B_2, \ldots, B_n]$ ，则 $A$ 与 $B$ 的夹角 $\theta$ 的余弦等于：

\begin{eqnarray}
\cos \theta & = & \frac{\Sigma^n_{i = 1} (A_i \times B_i)}
{\sqrt{\Sigma^n_{i = 1} (A_i)^2} \times \sqrt{\Sigma^n_{i = 1}
(B_i)^2}} \nonumber\\
& = & \frac{A \cdot B}{| A | \times | B |} \nonumber
\end{eqnarray}

这里的 $A_i$ 和 $B_i$ 分別代表向量 $A$ 和 $B$ 的各分量。

<sup><a id="fn.3" class="footnum" href="#fnr.3">3</a></sup> 这里使用 **Ochiai** 系数，或 **Ochiai-Barkman** 系数：

\begin{equation}
K = \frac{n(A \cap B)}{\sqrt{n(A) \times n(B)}} \nonumber
\end{equation}

这里 $A$ 和 $B$ 是集合， $n(A)$ 是 $A$ 的元素个数。如果集合由位向量所代表，那么可看到Ochiai系数跟余弦相似性是等同的。

<sup><a id="fn.4" class="footnum" href="#fnr.4">4</a></sup> 参见John S. Breese、David Heckerman和Carl Kadie的论文“Empirical Analysis of Predictive Algorithms for Collaborative Filtering” (Morgan Kaufmann Publishers，1998)。